In [2]:
import urllib.request
import datetime
import time
import os

if not os.path.isdir('data'):
    os.mkdir('data')

for i in range(1,28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean"
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now()
    date_and_time = now.strftime("%d-%m-%Y__%H-%M-%S")


    with open(f'data/id_{str(i)}_{date_and_time}.csv', 'wb') as f:
        f.write(text)
    print("saved file")

print("Done!")



saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
saved file
Done!


In [3]:
import os
import pandas as pd

def read_csv_files_to_df(directory):
    data_frames = []  # список для збереження Data


    for filename in os.listdir(directory):
        reg = filename.split("_")[1]
        file_path = os.path.join(directory, filename)

        try:
            data = pd.read_csv(file_path, index_col=False, header=1)
            data.columns = [col.strip().lower().replace("<br>", "") for col in data.columns]
            data = data.replace(to_replace=r'<.*?>', value='', regex=True)
            data["oblast"] = reg
            data_frames.append(data)

        except Exception as e:
            print(f"Не вдалося зчитати файл {file_path}: {e}")


    if data_frames:
        combined_data = pd.concat(data_frames, ignore_index=True)
    else:
        combined_data = pd.DataFrame()

    return combined_data

data = read_csv_files_to_df("data")
print(data.head())


   year  week    smn     smt    vci    tci    vhi oblast
0  1982   1.0  0.059  258.24  51.11  48.78  49.95     10
1  1982   2.0  0.063  261.53  55.89  38.20  47.04     10
2  1982   3.0  0.063  263.45  57.30  32.69  44.99     10
3  1982   4.0  0.061  265.10  53.96  28.62  41.29     10
4  1982   5.0  0.058  266.42  46.87  28.57  37.72     10


In [4]:
obl_dict = {
    1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька",
    5: "Житомирська", 6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська",
    9: "Київська", 10: "Кіровоградська", 11: "Луганська", 12: "Львівська",
    13: "Миколаївська", 14: "Одеська", 15: "Полтавська", 16: "Рівненська",
    17: "Сумська", 18: "Тернопільська", 19: "Харківська", 20: "Херсонська",
    21: "Хмельницька", 22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська",
    25: "Республіка Крим"
}

data["oblast"] = data["oblast"].astype(int).map(obl_dict)

In [10]:
#перетворюємо ствопчик років з рядка на ціле число
data["year"] = pd.to_numeric(data["year"], errors="coerce").astype("Int64")

print(data[(data["year"] == 2019)][:10])

      year  week    smn     smt    vci    tci    vhi          oblast
1924  2019   1.0  0.049  254.33  37.78  55.80  46.79  Кіровоградська
1925  2019   2.0  0.051  253.80  41.06  58.83  49.94  Кіровоградська
1926  2019   3.0  0.055  254.70  48.00  56.41  52.21  Кіровоградська
1927  2019   4.0  0.063  257.99  55.30  46.67  50.98  Кіровоградська
1928  2019   5.0  0.074  263.70  60.13  34.03  47.08  Кіровоградська
1929  2019   6.0  0.090  268.93  64.71  24.12  44.41  Кіровоградська
1930  2019   7.0  0.107  273.33  70.97  15.26  43.12  Кіровоградська
1931  2019   8.0  0.127  276.33  77.64  11.25  44.45  Кіровоградська
1932  2019   9.0  0.146  278.31  82.56  11.06  46.81  Кіровоградська
1933  2019  10.0  0.164  280.44  84.87  10.70  47.78  Кіровоградська


In [11]:
import statistics

def vhi_obl_year(obl, year):
    res = data[(data["oblast"] == obl) & (data["year"] == year)]
    res_vhi = list(res["vhi"])

    print(f"vhi для {obl} області на {year} рік склав \n{res_vhi}")

    print("\n\tMin:", min(res_vhi))
    print("\tMax:", max(res_vhi))

    print("\n\tAvg:", statistics.mean(res_vhi))
    print("\tMedian:", statistics.median(res_vhi))

vhi_obl_year("Черкаська", 2006)

vhi для Черкаська області на 2006 рік склав 
[50.31, 51.22, 49.33, 47.16, 45.88, 44.49, 41.57, 39.95, 43.31, 47.34, 49.36, 50.6, 50.5, 48.51, 46.96, 46.47, 48.97, 50.45, 49.51, 49.7, 51.02, 54.22, 57.03, 58.4, 58.14, 57.4, 55.8, 54.84, 54.8, 53.98, 54.46, 55.26, 55.67, 58.13, 60.7, 59.59, 57.71, 56.81, 54.87, 54.09, 52.81, 52.17, 51.26, 50.19, 48.38, 47.1, 47.57, 48.37, 48.96, 49.96, 50.68, 50.41]

	Min: 39.95
	Max: 60.7

	Avg: 51.39173076923077
	Median: 50.55


In [12]:
def vhi_obls_years(obl, min_year, max_year):
    res = data[(data["oblast"].isin(obl)) & (data["year"].between(min_year, max_year+1))]
    res_vhi = list(res["vhi"])

    print(f"vhi для {obl} області на {min_year}-{max_year} має {len(res_vhi)} значень")

    print("\n\tMin:", min(res_vhi))
    print("\tMax:", max(res_vhi))

    print("\n\tAvg:", statistics.mean(res_vhi))
    print("\tMedian:", statistics.median(res_vhi))

vhi_obls_years(["Черкаська", "Волинська", "Київська"], 1990, 2021)

vhi для ['Черкаська', 'Волинська', 'Київська'] області на 1990-2021 має 5148 значень

	Min: -1.0
	Max: 90.61

	Avg: 45.96346348096348
	Median: 45.88


In [13]:
def drought_years(percent=0):
    draught_years = []
    oblasti = data["oblast"].unique()
    min_obl_num = int(percent * len(oblasti) / 100)

    for year in data["year"].unique():
        draughts_in = 0
        for obl in oblasti:
            res = data[(data["oblast"] == obl) & (data["year"] == year)]
            res_vhi = list(res["vhi"])
            if len(res_vhi) != 0:
                if statistics.mean(res_vhi) < 35:
                    draughts_in += 1

        if draughts_in > min_obl_num:
            draught_years.append(int(year))

    print(f"Було {len(draught_years)} посушливих років")
    print(draught_years)

drought_years(20)

Було 3 посушливих років
[1985, 1986, 1994]
